In [190]:
import numpy as np
import random
from scipy.optimize import minimize, linear_sum_assignment
import itertools

In [191]:
# set up outfield player dictionaries
# array format: [defender, central, winger, forward]

all_players = dict()

# all_players['jazzie'] = [('D',1), ('C',0), ('W',0), ('F',0)]
# all_players['caroline'] = [('D',1), ('C',0), ('W',0), ('F',0)]
# all_players['helen'] = [('D',1), ('C',0), ('W',0), ('F',0)]
# all_players['christine'] = [('D',0), ('C',0), ('W',0.75), ('F',0.25)]
# all_players['sian'] = [('D',0), ('C',0), ('W',1), ('F',0)]
# all_players['vicky'] = [('D',0), ('C',0.5), ('W',0.25), ('F',0.25)]
# all_players['keah'] = [('D',0), ('C',0.25), ('W',0.5), ('F',0.25)]
# all_players['shafa'] = [('D',0), ('C',0), ('W',0.25), ('F',0.75)]
# all_players['olivia'] = [('D',0.1), ('C',0.6), ('W',0.1), ('F',0.2)]
# all_players['mychelle'] = [('D',0), ('C',0.5), ('W',0), ('F',0.5)]
# all_players['jaz'] = [('D',0.2), ('C',0.3), ('W',0.4), ('F',0.1)]

# all_players['jazzie'] = {'D':1, 'C':0, 'W':0, 'F':0} 
# all_players['caroline'] = {'D':1, 'C':0, 'W':0, 'F':0}
# all_players['helen'] = {'D':1, 'C':0, 'W':0, 'F':0}
# all_players['christine'] = {'D':0, 'C':0, 'W':0.75, 'F':0.25}
# all_players['sian'] = {'D':0, 'C':0, 'W':1, 'F':0}
# all_players['vicky'] = {'D':0, 'C':0.5, 'W':0.25, 'F':0.25}
# all_players['keah'] = {'D':0, 'C':0.25, 'W':0.5, 'F':0.25}
# all_players['shafa'] = {'D':0, 'C':0, 'W':0.25, 'F':0.75}
# all_players['olivia'] = {'D':0.1, 'C':0.6, 'W':0.1, 'F':0.2}
# all_players['mychelle'] = {'D':0, 'C':0.5, 'W':0, 'F':0.5}
# all_players['jaz'] = {'D':0.2, 'C':0.3, 'W':0.4, 'F':0.1}

# re-write as preferences
all_players['jazzie'] = [('D',1), ('C',0), ('W',0), ('F',0)]
all_players['caroline'] = [('D',1), ('C',0), ('W',0), ('F',0)]
all_players['helen'] = [('D',1), ('C',0), ('W',0), ('F',0)]
all_players['christine'] = [('D',0), ('C',0), ('W',1), ('F',2)]
all_players['sian'] = [('D',0), ('C',0), ('W',1), ('F',0)]
all_players['vicky'] = [('D',0), ('C',1), ('W',2), ('F',2)]
all_players['keah'] = [('D',0), ('C',2), ('W',1), ('F',2)]
all_players['shafa'] = [('D',0), ('C',0), ('W',2), ('F',1)]
all_players['olivia'] = [('D',4), ('C',1), ('W',3), ('F',2)]
all_players['mychelle'] = [('D',0), ('C',1), ('W',0), ('F',1)]
all_players['jaz'] = [('D',3), ('C',2), ('W',1), ('F',4)]

#convert preferences to costs
player_costs = dict()

for player in all_players.keys():
    prefs = all_players[player]

    weights = []
    for x in prefs:
        weight = 1-(1/x[1]) if x[1]!=0 else 1.0
        weights.append((x[0], weight))
        
    player_costs[player] = weights

In [192]:
def get_this_week_player_costs(this_week, player_costs=player_costs):
    this_week_player_costs = dict()
    for player in this_week:
        this_week_player_costs[player] = player_costs[player]
    return this_week_player_costs

def dict_to_cost_matrix(player_costs, formation):
    cost_matrix = []
    for player in player_costs:
        cost_list = []
        for position in formation:
            for x in player_costs[player]:
                if x[0]==position:
                    cost_list.append(x[1])
        cost_matrix.append(cost_list)
    
    return np.array(cost_matrix)

def append_subs_to_cost_matrix(cost_matrix, subs_weights):
    n_subs = len(cost_matrix) - 6
    cost_matrix_with_subs = []
    for i in range(len(cost_matrix)):
        cost_list = list(cost_matrix[i])
        for j in range(n_subs):
            cost_list.append(subs_weights[i])
        cost_matrix_with_subs.append(cost_list)
    return np.array(cost_matrix_with_subs)

def alter_subs_weights(cost_matrix, subs_weights):
    n_subs = len(cost_matrix) - 6
    cost_matrix_with_subs = []
    for i in range(len(cost_matrix)):
        cost_list = list(cost_matrix[i])
        for j in range(n_subs):
            cost_list[-(j+1)] = subs_weights[i] 
        cost_matrix_with_subs.append(cost_list)
    return np.array(cost_matrix_with_subs)

def convert_lsa_output_to_formation(cm, row_ind, col_ind, this_week, formation_with_subs):
    formation_dict = {'D':[], 'C':[], 'W':[], 'F':[], 'S':[]}
    for i in range(len(col_ind)):
        position = formation_with_subs[col_ind[i]]
        player = this_week[i]
        formation_dict[position].append(player)
    formation_dict['cost'] = cm[row_ind, col_ind].sum()
    return formation_dict
    

In [197]:
# build cost matrix
#this_week = ['jazzie', 'caroline', 'helen', 'sian', 'vicky', 'keah', 'olivia', 'shafa', 'jaz']
this_week = random.sample(all_players.keys(), 8)

print(this_week)
n_players_this_week = len(this_week)

formation = ['D', 'D', 'C', 'W', 'W', 'F']
formation_with_subs = formation.copy()
for i in range(n_players_this_week-6):
    formation_with_subs.append('S')

this_week_player_costs = get_this_week_player_costs(this_week)
cm_no_subs = dict_to_cost_matrix(this_week_player_costs, formation)

n_repeats = 0
if n_players_this_week == 7:
    n_repeats = 7
elif n_players_this_week == 8:
    n_repeats = 4
elif n_players_this_week == 9:
    n_repeats = 3

['jazzie', 'keah', 'christine', 'olivia', 'caroline', 'vicky', 'jaz', 'sian']


In [198]:
initial_subs_weights = np.zeros(n_players_this_week)
cm = append_subs_to_cost_matrix(cm_no_subs, initial_subs_weights)
    
subs_weights = initial_subs_weights.copy()
for i in range(n_repeats):
    row_ind, col_ind = linear_sum_assignment(cm)
    print(convert_lsa_output_to_formation(cm, row_ind, col_ind, this_week, formation_with_subs))
    
    for i in range(len(row_ind)):
        if formation_with_subs[col_ind[i]]=='S':
            subs_weights[i] += 2.0
    cm = alter_subs_weights(cm, subs_weights)
    
    #TODO: also alter position weights to maintain position?

{'D': ['jazzie', 'caroline'], 'C': ['olivia'], 'W': ['keah', 'christine'], 'F': ['vicky'], 'S': ['jaz', 'sian'], 'cost': 0.5}
{'D': ['jazzie', 'caroline'], 'C': ['olivia'], 'W': ['jaz', 'sian'], 'F': ['keah'], 'S': ['christine', 'vicky'], 'cost': 0.5}
{'D': ['jazzie', 'caroline'], 'C': ['vicky'], 'W': ['jaz', 'sian'], 'F': ['christine'], 'S': ['keah', 'olivia'], 'cost': 0.5}
{'D': ['olivia', 'jaz'], 'C': ['vicky'], 'W': ['keah', 'sian'], 'F': ['christine'], 'S': ['jazzie', 'caroline'], 'cost': 1.9166666666666667}


In [199]:
def objective_function(x, cm_no_subs, n_repeats):
    
    vals = []
    
    cm = append_subs_to_cost_matrix(cm_no_subs, x)
    subs_weights = x.copy()
    
    for i in range(n_repeats):
        row_ind, col_ind = linear_sum_assignment(cm)
        formation_dict = convert_lsa_output_to_formation(cm, row_ind, col_ind, this_week, formation_with_subs)
        vals.append(formation_dict['cost'])

        for i in range(len(row_ind)):
            if formation_with_subs[col_ind[i]]=='S':
                subs_weights[i] += 2.0
        cm = alter_subs_weights(cm, subs_weights)
    
    val = np.var(vals)
    #print(x, val)
    
    return val

this_week_player_costs = get_this_week_player_costs(this_week)
cm_no_subs = dict_to_cost_matrix(this_week_player_costs, formation)
#cm = append_subs_to_cost_matrix(cm_no_subs, np.zeros(n_players_this_week))

n_repeats = 0
if n_players_this_week == 7:
    n_repeats = 7
elif n_players_this_week == 8:
    n_repeats = 4
elif n_players_this_week == 9:
    n_repeats = 3
    
bound = (0, 1)
bounds = []
for i in range(n_players_this_week):
    bounds.append(bound)
    
res = minimize(objective_function, x0=np.zeros(n_players_this_week), args=(cm_no_subs, n_repeats), bounds=bounds, method='SLSQP', options={'eps':0.1})
print(res)



     fun: 0.0006063756186651453
     jac: array([0.06079841, 0.00458291, 0.00458291, 0.00458291, 0.06079841,
       0.00458291, 0.00555571, 0.00458291])
 message: 'Optimization terminated successfully'
    nfev: 43
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([5.89110402e-18, 6.61086081e-01, 6.61086081e-01, 6.61086081e-01,
       3.50104382e-18, 6.61086081e-01, 6.08182089e-01, 6.61086081e-01])


todo:
* set up cost matrix
* follow through to end of match
* if later combinations are forbidden, go back and adjust sub weights?
* could turn into a meta-optimisation problem, where you are trying to optimise the inital subs weights per player to minimise total cost to team
* integrate small weight reduction to incentivise not changing position
* dynamic cost matrix - can i prevent failure cases where algorithm wastes all best combinations early on?

In [200]:
initial_subs_weights = res.x
cm = append_subs_to_cost_matrix(cm_no_subs, initial_subs_weights)
    
subs_weights = initial_subs_weights.copy()
for i in range(n_repeats):
    row_ind, col_ind = linear_sum_assignment(cm)
    print(convert_lsa_output_to_formation(cm, row_ind, col_ind, this_week, formation_with_subs))
    
    for i in range(len(row_ind)):
        if formation_with_subs[col_ind[i]]=='S':
            subs_weights[i] += 2.0
    cm = alter_subs_weights(cm, subs_weights)

{'D': ['jazzie', 'caroline'], 'C': ['olivia'], 'W': ['keah', 'sian'], 'F': ['vicky'], 'S': ['christine', 'jaz'], 'cost': 1.7692681700749278}
{'D': ['jazzie', 'caroline'], 'C': ['olivia'], 'W': ['christine', 'jaz'], 'F': ['vicky'], 'S': ['keah', 'sian'], 'cost': 1.8221721621516895}
{'D': ['caroline', 'jaz'], 'C': ['olivia'], 'W': ['keah', 'sian'], 'F': ['christine'], 'S': ['jazzie', 'vicky'], 'cost': 1.8277527477425115}
{'D': ['jazzie', 'jaz'], 'C': ['vicky'], 'W': ['keah', 'sian'], 'F': ['christine'], 'S': ['olivia', 'caroline'], 'cost': 1.8277527477425115}
